<a href="https://colab.research.google.com/github/Bint7bara/KerasTunerProject/blob/master/keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl



In [ ]:
nf = 33
seed = 7
# load pima indians dataset
#dataset = np.loadtxt("NewGencode4DLTraining2.csv", delimiter=",")
dataset = np.loadtxt("12Featuresplus_LncAdeepScores.csv", delimiter=",")
#dataset = np.loadtxt("20LncAdeepScores.csv", delimiter=",")
dataset=dataset[~np.isnan(dataset).any(axis=1)]
# split into input (X) and output (Y) variables
X = dataset[:,0:nf]
print(type(X))
Y = dataset[:,nf]

<class 'numpy.ndarray'>


In [ ]:
from sklearn.preprocessing import StandardScaler
standardizer = StandardScaler()
X = standardizer.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val= train_test_split(
    X, Y, 
    test_size=0.2, random_state=42
)
X_train,y_train

(array([[ 2.45158971,  2.56959133,  1.56313942, ...,  0.8827685 ,
          0.88566565,  0.890931  ],
        [-0.15295742, -0.11225436, -0.97389834, ..., -1.27392751,
         -1.2332287 , -1.24491771],
        [-0.15623006, -0.17965225,  0.17462235, ...,  0.88412761,
          0.88943432,  0.88568057],
        ...,
        [-0.15522349, -0.17738426, -1.28313344, ..., -1.2664727 ,
         -1.25484382, -1.24009521],
        [-0.15623151, -0.1796575 , -0.58372879, ...,  0.78970569,
          0.82005118,  0.813293  ],
        [-0.15031626, -0.17767462, -0.1930914 , ...,  0.88900171,
          0.89209429,  0.89375365]]), array([1., 0., 1., ..., 0., 1., 1.]))

In [ ]:
import keras_tuner
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(1, activation="sigmoid"))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [ ]:
build_model(keras_tuner.HyperParameters())

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="KerasTuner",
)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [ ]:
tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val))
best_model = tuner.get_best_models()[0]

Trial 3 Complete [00h 05m 44s]
val_accuracy: 0.975980281829834

Best val_accuracy So Far: 0.9764935374259949
Total elapsed time: 00h 15m 19s


In [ ]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(X_train.shape)
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (19483, 33)               0         
                                                                 
 dense (Dense)               (19483, 384)              13056     
                                                                 
 dense_1 (Dense)             (19483, 1)                385       
                                                                 
Total params: 13,441
Trainable params: 13,441
Non-trainable params: 0
_________________________________________________________________


In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/KerasTuner
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 1
units_0: 384
activation: relu
dropout: False
lr: 0.004213313484484104
Score: 0.9764935374259949
Trial summary
Hyperparameters:
num_layers: 2
units_0: 288
activation: relu
dropout: True
lr: 0.0016467220069997504
units_1: 32
Score: 0.976288229227066
Trial summary
Hyperparameters:
num_layers: 2
units_0: 512
activation: relu
dropout: True
lr: 0.0001437116592547193
units_1: 160
Score: 0.975980281829834
